# MLflow KServe example integration
This example shows how to train a regressor with scikit. It stores the model object in MLflow and deploys the same object using KServe.

## Prerequisites

This example expects the Kubeflow and the MLflow bundle being deployed. Please refer to [this guide](https://documentation.ubuntu.com/charmed-mlflow/en/latest/tutorial/mlflow-kubeflow/). After the deployment of Kubeflow and MLflow please make these relations:

```
juju relate kserve-controller:service-accounts resource-dispatcher:service-accounts
juju relate kserve-controller:secrets resource-dispatcher:secrets
juju relate kserve-controller mlflow-minio
```

Following environment variables are expected to be set (in case of charmed kubeflow they are set for you):

* MLFLOW_S3_ENDPOINT_URL: endpoint for object storage 
* MLFLOW_TRACKING_URI: endpoint for mlflow server
* AWS_SECRET_ACCESS_KEY: secret key for object storage
* AWS_ACCESS_KEY_ID: username for object storage

In [20]:
!printenv | grep AWS

AWS_SECRET_ACCESS_KEY=2IY1YT1RT4A7KNU1GPA5GVGQPSSUOF
AWS_ACCESS_KEY_ID=minio


In [2]:
!printenv | grep MLFLOW

MLFLOW_UAT_PORT=tcp://10.152.183.235:80
MLFLOW_UAT_SERVICE_HOST=10.152.183.235
MLFLOW_S3_ENDPOINT_URL=http://mlflow-minio.kubeflow:9000
MLFLOW_UAT_SERVICE_PORT=80
MLFLOW_TRACKING_URI=http://mlflow-server.kubeflow.svc.cluster.local:5000
MLFLOW_UAT_PORT_80_TCP_PORT=80
MLFLOW_UAT_PORT_80_TCP_PROTO=tcp
MLFLOW_UAT_PORT_80_TCP=tcp://10.152.183.235:80
MLFLOW_UAT_PORT_80_TCP_ADDR=10.152.183.235
MLFLOW_UAT_SERVICE_PORT_HTTP_MLFLOW_UAT=80


## Setup (install dependencies)

In [ ]:
# pin the mlflow client to match the version of the deployed MLflow server
!pip install minio mlflow==2.1.1 boto3 tenacity -q

In [5]:
import mlflow
import numpy as np
import pandas as pd
import requests

from kubernetes import client as k8s_client, config as k8s_config
from mlflow.models.signature import infer_signature
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from tenacity import retry, stop_after_attempt, wait_exponential

## Download Data

In [6]:
data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [7]:
data.shape

(1599, 12)

## Preprocess Data

In [8]:
TARGET_COLUMN = "quality"
train, test = train_test_split(data)

train_x = train.drop([TARGET_COLUMN], axis=1)
test_x = test.drop([TARGET_COLUMN], axis=1)
train_y = train[[TARGET_COLUMN]]
test_y = test[[TARGET_COLUMN]]


## Create MLflow experiment

In [9]:
wine_experiment_name = "My Wine Experiment Kserve"
experiment = mlflow.get_experiment_by_name(wine_experiment_name)
experiment_id = (
    mlflow.create_experiment(name=wine_experiment_name)
    if experiment is None
    else experiment.experiment_id
)

In [10]:
# check that the experiment was created successfully
assert mlflow.get_experiment(experiment_id).name == wine_experiment_name, f"Failed to create experiment {wine_experiment_name}!"

## Train and store model

In [11]:
def experiment(alpha, l1_ratio):
    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name='wine_models', experiment_id=experiment_id) as run:
            mlflow.set_tag("author", "kf-testing")
            lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
            lr.fit(train_x, train_y)

            pred_y = lr.predict(test_x)
            mlflow.log_metric("rmse", np.sqrt(mean_squared_error(test_y, pred_y)))
            mlflow.log_metric("r2", r2_score(test_y, pred_y))
            mlflow.log_metric("mae", mean_absolute_error(test_y, pred_y))

            signature = infer_signature(test_x, pred_y)
            result = mlflow.sklearn.log_model(lr, "model", registered_model_name="wine-elasticnet", signature=signature)
            model_uri = f"{mlflow.get_artifact_uri()}/{result.artifact_path}"
    
    return run, model_uri

In [12]:
run, model_uri = experiment(0.5, 0.5)

2023/10/26 11:31:46 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmphgdkiwyh/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==0.24.2', 'cloudpickle==1.6.0']. Set logging level to DEBUG to see the full traceback.
2023/10/26 11:31:46 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2023/10/26 11:31:48 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp9m2mw9c2/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==0.24.2', 'cloudpickle==1.6.0']. Set logging level to DEBUG to see the full traceback.
Registered model 'wine-elasticnet' already exists. Creating a new version of this model...
2023/10/26 11:31:48 INFO mlflow.tracking._model_registry.client: Waiting up 

In [13]:
model_uri

's3://mlflow/1/fe0d84c6313f4853bcb83e5fe7cbe39a/artifacts/model'

## Deploy Kserve's InferenceService

In [ ]:
!pip install kserve kubernetes -q

In [15]:
from kubernetes.client import V1ObjectMeta
from kubernetes import client as k8s_client, config as k8s_config
from kserve import (
    constants,
    KServeClient,
    V1beta1InferenceService,
    V1beta1InferenceServiceSpec,
    V1beta1PredictorSpec,
    V1beta1SKLearnSpec,
)

In [16]:
ISVC_NAME = "wine-regressor3"

isvc = V1beta1InferenceService(
    api_version=constants.KSERVE_V1BETA1,
    kind=constants.KSERVE_KIND,
    metadata=V1ObjectMeta(
        name=ISVC_NAME,
        annotations={"sidecar.istio.io/inject": "false"},
    ),
    spec=V1beta1InferenceServiceSpec(
        predictor=V1beta1PredictorSpec(
            service_account_name="kserve-controller-s3",
            sklearn=V1beta1SKLearnSpec(
                storage_uri=model_uri
            )
        )
    )
)

In [17]:
client = KServeClient()
client.create(isvc)

@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(30),
    reraise=True,
)
def assert_isvc_created(client, isvc_name):
    """Wait for the Inference Service to be created successfully."""
    assert client.is_isvc_ready(ISVC_NAME), f"Failed to create Inference Service {isvc_name}."

assert_isvc_created(client, ISVC_NAME)

## Run Prediction

In [18]:
isvc_resp = client.get(ISVC_NAME)
isvc_url = isvc_resp['status']['address']['url']
print("Inference URL:", isvc_url)

Inference URL: http://wine-regressor3.user123.svc.cluster.local


In [19]:
inference_input = {
    "instances": [
        [10.1, 0.37, 0.34, 2.4, 0.085, 5.0, 17.0, 0.99683, 3.17, 0.65, 10.6]
    ]
}
response = requests.post(f"{isvc_url}/v1/models/{ISVC_NAME}:predict", json=inference_input)
print(response.text)

{"predictions":[5.759254882639977]}


## Delete Inference Service

In [22]:
client.delete(ISVC_NAME);